In [1]:
import pandas as pd
import numpy as np


from pyiron import Project, ase_to_pyiron
from pyiron_contrib.atomistic.atomicrex.atomicrex_job import Atomicrex

In [2]:
df = pd.read_pickle("Cu_df1_A1_A2_A3_EV_elast_phon.pckl.gzip", compression="gzip")

In [3]:
pr = Project("WorkshopPotential")
pr.remove_jobs()

Are you sure you want to delete all jobs from 'WorkshopPotential'? y/(n) y


In [4]:
job = pr.create_job(Atomicrex, "PotentialDF1", delete_existing_job=True)

### Add the structures that should be fitted.
It is possible to assign different weights to certain structures or properties, depending on what should be investigated using the potential. Here every structure has the same weight, but the force vector with N*3 values is normalized to have the same total weight as the single value energy. Therefore it is divided by the number of atoms. 

In [5]:
for id, row in df.iterrows():
    struct = ase_to_pyiron(row.ase_atoms)
    s = job.structures.add_structure(struct, f"id{id}", relative_weight=1)
    s.fit_properties.add_FitProperty("atomic-energy", target_value=row.energy_corrected_per_atom , relative_weight=1)
    s.fit_properties.add_FitProperty("atomic-forces", target_value=row.forces, relative_weight=1/row.NUMBER_OF_ATOMS)

### Define the type of potential and necessary functions.
In this case an eam potential is fitted.

In [6]:
job.potential = job.factories.potentials.eam_potential()

It is necessary to define a pair potential, an electronic density function and an embedding function.
For all of those it is possible to choose between different functional forms.
Classic pair potentials are physically motivated and  have a very limited number of paramaters that are derived from a experimentally measured quantity.
Splines or polynomials offer more flexibility, but are not directly physically motivated and can lead to unphysical oscillations or overfitting.

In this case a generalized morse function is used for the pair interaction, while the electronic density and embedding function will be splines. Depending on the properties that are calculated other functional forms could give better results.

The parameters in D0=3.5 and r0=1.8 are the approximate cohesive energy and the equilibrium lattice constant. Beta and S can also be derived from physical quantities but are chosen randomly in a typical range in this case. Delta is a parameter that shifts the whole function up or down. The initial parameter choices should not matter too much as long as they are somewhat reasonable since they will be optimized in the fitting process anyway.

In [7]:
V = job.factories.functions.morse_B(identifier="V_CuCu", D0=3.5, r0=1.8, beta=2, S=2, delta=0)

Additionally a screening function needs to be defined for the morse potential

In [8]:
V.screening  = job.factories.functions.exp_A_screening(identifier="V_cutoff", cutoff=7)

The electron density is chosen to be a spline function. The cutoff has to be defined. Derivatives left and right are optional, they default to 0. For the right cutoff this is fine, since the forces should smoothly go to 0. For the left this is not necessarily the best choice, since the function value should increase at very close distances.

In [9]:
rho = job.factories.functions.spline(identifier="rho_CuCu", cutoff=7, derivative_left=-1)

For a spline function it is necessary to define node points. They can be equally spaced or sampled with higher density around turning points, f.e. the first neighbor distance.
Too few nodepoints mean low flexibilty, too many lead to overfitting. This requires some testing to find an optimal choice.

In [10]:
rho_nodes = np.array([0.5, 1, 2, 3, 4, 5.5, 7])

The nodes need initial values. The electron density should be proportional to $e^{-r}$, so this function is chosen to calculate them.

In [11]:
decaying_exp = lambda r: np.exp(-r)
rho_initial = decaying_exp(rho_nodes)

Additionally it is a good idea to define limits for the node points. This is optional for local minimizers, but the fit can quickly run away without limits. Global optimizers typically require them to constrain the sampled space.

A density can't be negative so the lower limit is set to 0. The upper limit is chosen to be 3 times the initial values. These choices aswell as the choice for $e^{-r}$ as initial values are somewhat arbitrary, but don't matter much. The electron density from single atoms does not directly influence the calculated energies and forces, instead the summed up density at some place is used in the embedding function, so the final numerical values are an interplay between electron density and embedding function. Since the latter will also be a spline function it can only be defined for a certain range of rho values as node points. Therefore it is better to limit the range of electron density values and define larger limits for the embedding function instead. 

In [12]:
rho_mins = np.zeros((len(rho_nodes)))
rho_maxs = 3*rho_initial
rho.parameters.create_from_arrays(rho_nodes, rho_initial, min_vals=rho_mins, max_vals=rho_maxs)

In [13]:
rho.parameters["node_7.0"].start_val = 0
rho.parameters["node_7.0"].enabled = False

$-\sqrt(\rho)$ can be used as initial guess for the embedding energy, which is taken from second moment approximation tight binding. 
The node points have to be chosen in a range compatible to the electron density. This can be estimated by calculating it for a densely packed structure.
Alternatively atomicrex writes the maximum electron density of all structures to the output. This can be used as a hint for the node points for consequent fits.
Everything else is similar to the electron density.

In [14]:
F = job.factories.functions.spline(identifier="F_CuCu", cutoff=5)
F_nodes = np.array([0.0, 1, 1.5, 2, 2.5, 3, 3.5, 4, 5])
F_init = -np.sqrt(F_nodes)
F_maxs = np.zeros(len(F_nodes))
F_mins = -np.ones(len(F_nodes))*5
F.parameters.create_from_arrays(F_nodes, F_init, F_mins, F_maxs)
F.parameters["node_0.0"].enabled=False
F.parameters["node_0.0"].start_val = 0
F.parameters["node_5.0"].enabled=False
F.parameters["node_5.0"].start_val = 0

The functions have to be assigned to the potential

In [15]:
job.potential.pair_interactions[V.identifier] = V
job.potential.electron_densities[rho.identifier] = rho
job.potential.embedding_energies[F.identifier] = F

### Define fitting procedure
Finally a few parameters need to be set that influence the fitting process.
The minimization can be done with different algorithms. Atomicrex itself implements the BFGS algorithm. Additionally the algorithms from the nlopt library can be used.

In [16]:
## Define the atom types of the potential
job.input.atom_types.Cu = None
##
job.input.fit_algorithm = job.factories.algorithms.ar_lbfgs(max_iter=500000)
job.server.cores = 8
job.run()
job.output

The job PotentialDF1 was saved and received the ID: 616


Output({'error': None, 'residual': 0.827093, 'iterations': 8326})

### Same for the 1000 structures dataset
### This can take long and writes 1000 seperate POSCAR files, TAKE CARE

In [17]:
j = pr.create_job(Atomicrex, "PotentialDF2", delete_existing_job=True)
j.potential = job.potential.copy()
j.input = job.input.copy()

In [18]:
df = pd.read_pickle("Cu_df2_1k.pckl.gzip", compression="gzip")
for id, row in df.iterrows():
    struct = ase_to_pyiron(row.ase_atoms)
    s = j.structures.add_structure(struct, f"id{id}", relative_weight=1)
    s.fit_properties.add_FitProperty("atomic-energy", target_value=row.energy_corrected_per_atom , relative_weight=1)
    s.fit_properties.add_FitProperty("atomic-forces", target_value=row.forces, relative_weight=1/row.NUMBER_OF_ATOMS)

In [19]:
import time

In [20]:
j.server.cores = 16
t1 = time.time()
j.run()
t2 = time.time()

The job PotentialDF2 was saved and received the ID: 617


In [21]:
t2-t1

176.67261242866516

In [22]:
j.output

Output({'error': None, 'residual': 7948.44, 'iterations': 825})